In [1]:
cd /Data1/hmd2/notebooks_th/Fix_models/Attn_lstm_with_sch/

/Data1/hmd2/notebooks_th/Fix_models/Attn_lstm_with_sch


In [2]:
import os
import pathlib
import numpy as np
import scipy.io as spio
from scipy.signal import resample
import librosa
import IPython.display as ipd
import torch

import config

In [3]:
data_folder = pathlib.Path("/Data2/murmur/train")

In [4]:
wav_list = [i for i in os.listdir(data_folder) if i.endswith(".wav")]
len(wav_list)

2532

In [5]:
sample_filepath = wav_list[0]
fs, recording = spio.wavfile.read(data_folder / sample_filepath)

In [6]:
fs

4000

In [7]:
# 원본 오디오 데이터 품질

ipd.display(ipd.Audio(recording, rate= 4000))

In [8]:
# 리샘플링시 목표 샘플링레이트 설정
sr = 4000 # 기존 샘플링레이트
new_sr = 2000 # 목표 샘플링레이트

# 다운샘플링할 샘플 수 계산
num_samples = int(len(recording) * new_sr / sr)

# 리샘플링
audio_resampled = resample(recording, num_samples)

In [9]:
audio_resampled_int = np.int16(audio_resampled / np.max(np.abs(audio_resampled)) * 32767)

In [10]:
# 이유는 모르겠지만... 리샘플링 한 이후에는 청음기능이 작동되지 않음.
ipd.display(ipd.Audio(audio_resampled_int, rate= 2000))

In [11]:
# 따라서 리샘플링한 데이터를 .wav파일로 저장 후 청음시도
## 기존과 리샘플링한 데이터 간의 품질에 큰 차이가 없다고 판단되었음.

from scipy.io import wavfile 

wavfile.write("/Data1/hmd2/notebooks_th/notes/resampled_audio.wav", new_sr, audio_resampled_int)

In [17]:
sample_wav = "85345_PV.wav"

sr, recording = spio.wavfile.read(data_folder / sample_wav)

In [18]:
len(recording)

73024

In [20]:
num_samples

146048

In [22]:
sample_wav = "85345_PV.wav"

sr, recording = spio.wavfile.read(data_folder / sample_wav)
new_sr = 2000

num_samples = int(len(recording) * new_sr / sr)
recording = resample(recording, num_samples)
recording = torch.as_tensor(recording.copy())

# calculate_features
recording = recording.float()
recording -= recording.mean()
recording /= recording.abs().max()

window_length = int(config.window_length * fs) 
window_step = int(config.hop_length * fs)

print(window_length, window_step)

spectrogram = (
            torch.stft(
                recording,
                n_fft=window_length,
                hop_length=window_step,
                window=torch.hann_window(window_length),
                center=False,
                return_complex=False,
            )
            .pow(2)
            .sum(dim=-1)
        )

spectrogram = spectrogram[: 40]
spectrogram = torch.log(spectrogram)
spectrogram = (spectrogram - spectrogram.mean(dim=-1, keepdims=True)) / spectrogram.std(dim=-1, keepdims=True)

features_fs = int(1 / config.hop_length)


spectrogram.shape

200 80


torch.Size([40, 454])